# WebScarping 

Modified Original Script From https://github.com/shamafarabi/NLP-Predict-Cast-Member-of-the-TV-Show-Friends

 # Import Packages

In [ ]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
from collections import defaultdict
import re
import json

In [ ]:
url = "https://fangj.github.io/friends/"
html = urlopen(url)

In [ ]:
soup = BeautifulSoup(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [ ]:
# Print out the text
text = soup.get_text()

In [ ]:
season_list = text.split('\n')

In [ ]:
print(season_list[-20:])

['1001 Joey and Rachel Kiss', '1002 Ross Is Fine', "1003 Ross's Tan", '1004 The Cake', "1005  Rachel's Sister Babysits", "1006  Ross's Grant", '1007 The Home Study', '1008 The Late Thanksgiving', '1009 The Birth Mother', '1010 Chandler Gets Caught', '1011 The Stripper Cries', "1012  Phoebe's Wedding", '1013 Joey Speaks French', '1014 Princess Consuela', '1015 Estelle Dies', "1016 Rachel's Going Away Party", '1017-1018 The Last One, Part I & II', '', '', '']


In [ ]:
episodes =[l for l in season_list if l[0:3].isdigit()];
episode_numbers, episode_names = [], []
for l in episodes:
    matches = re.match(r'^(?:(\d+-\d+)|(\d+))\s(.+)$', l)
    if matches.group(1):
        episode_numbers.append(matches.group(1))
    else:
        episode_numbers.append(matches.group(2))
    episode_names.append(matches.group(3).strip())

In [ ]:
print(episode_numbers)

['101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '201', '202', '203', '204', '205', '206', '207', '208', '209', '210', '211', '212-213', '214', '215', '216', '217', '218', '219', '220', '221', '222', '223', '224', '301', '302', '303', '304', '305', '306', '307', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '401', '402', '403', '404', '405', '406', '407', '408', '409', '410', '411', '412', '413', '414', '415', '416', '417', '418', '419', '420', '421', '422', '423', '501', '502', '503', '504', '505', '506', '507', '508', '509', '510', '511', '512', '513', '514', '515', '516', '517', '518', '519', '520', '521', '522', '523', '601', '602', '603', '604', '605', '606', '607', '608', '609', '610', '611', '612', '613', '614', '615-616', '617', '618', '619', '620', '621', '622', '623', '624', '701

Except for season 724, Most common url formats are addressed as 3 digit or 7 digit format. we will do try and except for that one episode so that we can continue the loop

In [ ]:
all_transcripts =   {'Episodes':[], 'Episode_Names':[],'Characters': [],'Lines': []} 
for episode in episode_numbers:
    try:
        # to extract url that has 3 digit as the number of the episode (e.g. 101)
        episode_url = episode
        if len(episode_url) in [3, 4]:
            if len(episode) == 3:
                episode_url = '0' + episode_url
        else:
            # to extract url that has 7 digit as the number of the episode (e.g. 212-213)
            if len(episode_url) == 7:
                episode_url = '-'.join(map(lambda x: '0' + x, episode_url.split('-')))
        url = 'https://fangj.github.io/friends/season/{0}.html'.format(episode_url)
        html = urlopen(url)
    # only one  url  of all the episodes had a diferent format than the above two in the url, ignore it
    except Exception as e:
        continue
            
    #parse the html
    soup = BeautifulSoup(html, 'html.parser')
    #identify html tags and attributes associated with actors and their dialogues in the script
    transcripts = soup.find_all('p')

     # Print text only, and further filter tags to fin Charcater: Lines
    pattern = '^\w+[:]'   
    transcripts_text_only = [p.text for p in transcripts if re.search(pattern,p.text) ]

    #split Characters and Lines
    Characters_this_episode = [l.split(":") [0] for l in transcripts_text_only]
    Lines_this_episode = [l.split(":") [1] for l in transcripts_text_only]
    
    #store episode number and episode name for each line
    Episode = [episode]*len(Characters_this_episode) 
    Episode_Name = [episode_names[episode_numbers.index(episode)]]*len(Characters_this_episode)
    
    all_transcripts['Episodes'].append(Episode)
    all_transcripts['Episode_Names'].append(Episode_Name)
    all_transcripts['Characters'].append(Characters_this_episode)
    all_transcripts['Lines'].append(Lines_this_episode)

In [ ]:
type(all_transcripts['Episodes'])

list

In [ ]:
#Each dictionary has list of lists as values. Flatten them so that each key has a single list
import itertools
all_transcripts['Episodes']= list (itertools.chain.from_iterable(  all_transcripts['Episodes']))
all_transcripts['Episode_Names']= list (itertools.chain.from_iterable(  all_transcripts['Episode_Names']))
all_transcripts['Characters']= list (itertools.chain.from_iterable(  all_transcripts['Characters']))
all_transcripts['Lines']= list (itertools.chain.from_iterable(  all_transcripts['Lines']))


In [ ]:
print(len(all_transcripts['Episodes']))
print(len(all_transcripts['Episode_Names']))
print(len(all_transcripts['Characters']))
print(len(all_transcripts['Lines']))

53513
53513
53513
53513


# Export dictionary of transcripts as CSV

In [ ]:
csv_df = pd.DataFrame(all_transcripts)

In [ ]:
csv_df.Lines = csv_df.Lines.apply(lambda x: ' '.join(x.split()))

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\(.*?\)\s', ' ', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\(.*?\)$', ' ', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\(.*?\)\.\.\.', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\(.*?\)', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\(.*?$', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\{.*?\)', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'(?<=\.).+?(?<!\d)\)', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'(?<=\?).+?\)', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\[.*?\]\s', ' ', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\{.*?\}\s', ' ', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\[.*?$', ' ', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\{.*?\}$', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\s\{.*?$', ' ', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.replace(r'\[.*?\]', '', regex=True)

In [ ]:
csv_df.Lines = csv_df.Lines.str.strip()

In [ ]:
csv_df = csv_df[~(csv_df.Lines.str.len() == 0)]

In [ ]:
csv_df.Characters = csv_df.Characters.str.upper()
csv_df.Characters = csv_df.Characters.str.strip()

In [ ]:
csv_df.Characters = csv_df.Characters.str.replace(r'^RACH$', 'RACHEL', regex=True)
csv_df.Characters = csv_df.Characters.str.replace(r'^MNCA$', 'MONICA', regex=True)
csv_df.Characters = csv_df.Characters.str.replace(r'^PHOE$', 'PHOEBE', regex=True)
csv_df.Characters = csv_df.Characters.str.replace(r'^CHAN$', 'CHANDLER', regex=True)
csv_df.Characters = csv_df.Characters.str.replace(r'^RACHE$', 'RACHEL', regex=True)
csv_df.Characters = csv_df.Characters.str.replace(r'^RACEL$', 'RACHEL', regex=True)
csv_df.Characters = csv_df.Characters.str.replace(r'^CHANDLERS$', 'CHANDLER', regex=True)

In [ ]:
csv_df.to_csv('Friends_transcripts.csv', index=False)